In [1]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `C:\Users\lukas\.julia\environments\v1.9`


In [2]:
Pkg.status()

Status `C:\Users\lukas\.julia\environments\v1.9\Project.toml`
  [87dc4568] HiGHS v1.7.5
  [7073ff75] IJulia v1.24.2
⌃ [4076af6c] JuMP v1.16.0
Info Packages marked with ⌃ have new versions available and may be upgradable.


In [3]:
using JuMP, HiGHS

# Description of the Problem 

A manager of a VVP is interested in maximizing his profit. His VVP consists of the following components:

*  A dispatchable power plant that burn fossil fuels to produce electricity
* Flexible loads which are owned by the VVP
* Storage equipment 
* A stochastic generating unit that produce energy using wheater-driven energy sources

The VVP manager wants to know how much energie $Pd$ he should sell to the market in order to maximize his profit. But he wants to take the uncertainty of his wind turbine into account. Therefore, he
considers two scenarios $w1$ and $w2$ and three consectutive time intervalls $T= t1, t2 ,t3 $ for his operation. His analysts estimate that the market clearing price $λ= 20$ is constant for both scenarios and all times $T$. 

In [4]:
λ= 20  #market clearing price
w=[0.2, 0.8] #probabilities for three different scenarios
Pw1=[0, 200, 300] #Power output wind turbine for Scencario 1 for t=1 to t=3
Pw2=[50, 210, 200] #Power output wind turbine for Scencario 2 for t=1 to t=3

#initalize Model
profit = Model(HiGHS.Optimizer)
#Pd amount of energie to sell to the market for t=1 to t=3
@variable(profit, Pd[1:3])

3-element Vector{VariableRef}:
 Pd[1]
 Pd[2]
 Pd[3]

## Dispatchable Power Plants

The VVP Manager has one coal-fired power station $Pg$ which produces energie $E_{G}$ within the range $E_{G}^{min} \le E_{G} \le E_{G}^{max}$.
Since the power plant is running on fossil resources, the power plant has marginal costs of 25 dollars and is described by the cost function: $C_{G}(E_{G})$

$C_{i}(E_{Gi})$ Cost function for generating amount of energy $E_{Gi}$
<br/>

This gives us the following variables:$

In [5]:
Pg_min=180 #min power output of powerplant Pg
Pg_max=440 #max power output of powerplant Pg

Cg= 25 #marginal costs of producing one unit of energy for powerplant Pg

@variable(profit, Pg_min <= Pg_w1[1:3] <= Pg_max ) #define dispatchable pp output for scenario 1 (w1) for t=1 to t=3
@variable(profit, Pg_min <= Pg_w2[1:3] <= Pg_max ) #define dispatchable pp output for scenario 2 (w2) for t=1 to t=3

3-element Vector{VariableRef}:
 Pg_w2[1]
 Pg_w2[2]
 Pg_w2[3]

## Flexible loads

The VVP Manager has the option to use the produced energie for his small hydrogen production plant $El$ instead of selling it to the market. By selling the hydrogen, he gets a utility $Ul$ of 17 dollars. However, his buyers do not always need the same amount of energie, so $El$ is limited to $0 \le El \le El^{max}$ for each $T$
<br/>
<br/>
This gives us the following variables:

In [6]:
Ul=17 #utility from using the produced energy for the flexible load
El_max=[100, 50, 30] #maximum for flexible loads

@variable(profit, 0<= El_w1[i in keys(El_max)] <= El_max[i]) #define flexible load output for scenario 1 (w1) for t=1 to t=3
@variable(profit, 0<= El_w2[i in keys(El_max)] <= El_max[i]) #define flexible load output for scenario 2 (w2) for t=1 to t=3

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, [1, 2, 3]
And data, a 3-element Vector{VariableRef}:
 El_w2[1]
 El_w2[2]
 El_w2[3]

## Storage Equipment

The VVP Manager has also a storing unit $Ps$ where he can charge and discharge energie at any time $T$. However, the storage $Es$
is limited to $SOC^{min} \le Es \le SOC^{max}$ and the unit charges $Psc$ and discharges $Psd$ only at a certain rate $Psc^{min} \le Psc \le Psc^{max}$ ;
$Psd^{min} \le Psd \le Pscd^{max}$ with a given charging efficiency $nc = 0.95$ and discharging efficiency $nd = 0.9$ 

This gives us the following variables and constraints:

In [7]:
SOC_max= 300 #max capacity of the battery
SOC_min= 100 #max capacity of the battery

Psd_max= 0.2 * SOC_max #max decharging rate for the battery
Psc_max= 0.2 * SOC_max #max charging rate for the battery

nc= 0.95 #charging efficiency 
nd = 0.9 #discharging efficiency

#maximum charging and discharge rate for w1 for all T
@variable(profit, 0<= Psc_w1[1:3] <= Psc_max) 
@variable(profit, 0<= Psd_w1[1:3] <= Psd_max)

#maximum charging and discharge rate for w2 for all T
@variable(profit, 0<= Psc_w2[1:3] <= Psc_max)
@variable(profit, 0<= Psd_w2[1:3] <= Psd_max)

#amount of enegerie whtinn storage unit  in w1 for all T
@variable(profit, SOC_min <= Es_w1[1:3] <= SOC_max)
@constraint(profit, Es_w1[1] == 100 + nc * Psc_w1[1] - 1/nd * Psd_w1[1])
@constraint(profit, Es_w1[2] == Es_w1[1] + nc * Psc_w1[2] - 1/nd * Psd_w1[2])
@constraint(profit, Es_w1[3] == Es_w1[2] + nc * Psc_w1[3] - 1/nd * Psd_w1[3])

@variable(profit, SOC_min <= Es_w2[1:3] <= SOC_max) #amount of enegerie whtinn storage unit  in w2 for all T
@constraint(profit, Es_w2[1] == 100 + nc * Psc_w2[1] - 1/nd * Psd_w2[1])
@constraint(profit, Es_w2[2] == Es_w2[1] + nc * Psc_w2[2] - 1/nd * Psd_w2[2])
@constraint(profit, Es_w2[3] == Es_w2[2] + nc * Psc_w2[3] - 1/nd * Psd_w2[3])

-0.95 Psc_w2[3] + 1.1111111111111112 Psd_w2[3] - Es_w2[2] + Es_w2[3] == 0

## Stochastic generating units

The VVP modeled the power output for his wind turbine $Pw$ for two scenarios depending on the wind conditions at the given time T. However, 
the manager has the option to curtail the production $Pw^{curt}$

This gives us the following variables:

In [8]:
Pw1=[0, 200, 300] #Scencario 1 for t=1 to t=3
Pw2=[50, 210, 200] #Scencario 2 for t=1 to t=3

@variable(profit, 0<= Pw1_curt[i in keys(El_max)] <= Pw1[i])
@variable(profit, 0<= Pw2_curt[i in keys(El_max)] <= Pw2[i])

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, [1, 2, 3]
And data, a 3-element Vector{VariableRef}:
 Pw2_curt[1]
 Pw2_curt[2]
 Pw2_curt[3]

## Ohter constraints 

Besides the constraints of the varialbes, the amount of energie produced and consumed has to be at balance at any given point in time $T$
$\sum_{i} E_Giw (t) + \bigg(\sum \hat{P}_{W_qw} (t) -P^{curt}_{Wqw}(t)+ \sum_{k} P^d_{Skw} \bigg) = \sum_{j} E_{L_jw} (t) + 
\bigg (\sum_{k} P^c_{Skw} + P^D (t) \bigg)$

This gives us the constraints:

In [9]:
for g in keys(El_max)
    @constraint(profit, Pd[g] == Pg_w1[g] + Pw1[g] + Psd_w1[g] - Psc_w1[g] - El_w1[g] -Pw1_curt[g])
end
for g in keys(El_max)
    @constraint(profit, Pd[g] == Pg_w2[g] + Pw2[g] + Psd_w2[g] - Psc_w2[g] - El_w2[g] -Pw2_curt[g])
end

## Formulating Objective Function

Given the formulated variables and constraints the VPP manager can formulate his objective function:
<br/>
$\max\sum_{t=1}^{T} \bigg(\lambda^D (t).P^D (t) - \sum_{i} C_i(E_Gi(t)) + \sum U_j(E_Gj(t))\bigg)$
    

In [10]:
@objective(profit, Max, 
        sum(Pd[i]*λ for i in keys(Pd))+
        w[1] * (
        -sum(Pg_w1[i]*Cg for i in keys(Pg_w1)) 
        +sum(El_w1[i]*Ul for i in keys(El_w1)) 
        )+   
        w[2] *(
        -sum(Pg_w2[i]*Cg for i in keys(Pg_w2)) 
        +sum(El_w2[i]*Ul for i in keys(El_w2)) 
        )
)

20 Pd[1] + 20 Pd[2] + 20 Pd[3] - 5 Pg_w1[1] - 5 Pg_w1[2] - 5 Pg_w1[3] + 3.4000000000000004 El_w1[1] + 3.4000000000000004 El_w1[2] + 3.4000000000000004 El_w1[3] - 20 Pg_w2[1] - 20 Pg_w2[2] - 20 Pg_w2[3] + 13.600000000000001 El_w2[1] + 13.600000000000001 El_w2[2] + 13.600000000000001 El_w2[3]

## Overview of the objective function and constraints

In [11]:
print(profit)

## Optimizing the function

In [12]:
optimize!(profit)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
12 rows, 38 cols, 57 nonzeros
12 rows, 38 cols, 57 nonzeros
Presolve : Reductions: rows 12(-0); columns 38(-1); elements 57(-1)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -6.0000000000e+04 Ph1: 6(6000); Du: 3(60) 0s
         15    -6.4330877193e+03 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 15
Objective value     :  6.4330877193e+03
HiGHS run time      :          0.00


In [13]:
for i in keys(Pd) 
    print(" value= ", value(Pd[i]))
end

 value= 180.0 value= 369.82456140350877 value= 440.0

In [14]:
for variable in all_variables(profit)
    println(string(name(variable), " = ", value(variable)))
end

Pd[1] = 180.0
Pd[2] = 369.82456140350877
Pd[3] = 440.0
Pg_w1[1] = 180.0
Pg_w1[2] = 180.0
Pg_w1[3] = 180.0
Pg_w2[1] = 180.0
Pg_w2[2] = 180.0
Pg_w2[3] = 180.0
El_w1[1] = 0.0
El_w1[2] = 10.175438596491233
El_w1[3] = 30.0
El_w2[1] = 0.0
El_w2[2] = 0.0
El_w2[3] = 0.0
Psc_w1[1] = 0.0
Psc_w1[2] = 0.0
Psc_w1[3] = 0.0
Psd_w1[1] = -0.0
Psd_w1[2] = -0.0
Psd_w1[3] = 0.0
Psc_w2[1] = 50.0
Psc_w2[2] = 20.175438596491247
Psc_w2[3] = 0.0
Psd_w2[1] = 0.0
Psd_w2[2] = 0.0
Psd_w2[3] = 60.0
Es_w1[1] = 100.0
Es_w1[2] = 100.0
Es_w1[3] = 100.0
Es_w2[1] = 147.5
Es_w2[2] = 166.66666666666669
Es_w2[3] = 100.0
Pw1_curt[1] = 0.0
Pw1_curt[2] = 0.0
Pw1_curt[3] = 10.0
Pw2_curt[1] = 0.0
Pw2_curt[2] = 0.0
Pw2_curt[3] = 0.0


The solver delivered a optimal solution. The VVP Manager can expect to make a profit of approximately 6433 dollars. 